In [1]:
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostRegressor
from algolab import API
from datetime import datetime
from config import *
import pandas as pd, numpy as np, json, os

import yfinance as yf

In [2]:
son=pd.read_excel("./excel/FiyatPenceresi.Xlsx")
endeks=son.iloc[-1,:]
son=son.iloc[:-1,:]
hisseler=son["Kod"].values
hisseler=np.array(hisseler)
for i in range(len(hisseler)):
    hisseler[i]=hisseler[i]+".IS"

In [5]:
df=pd.DataFrame()
for hisse in hisseler:
    hisse_verileri = yf.download(hisse,start="2010-01-01",end="2024-06-11")  # Örnek tarih aralığı
    hisse_kapanis = hisse_verileri[['High','Low','Open','Close','Volume']]
    hisse_kapanis["Hisse"]=hisse
    hisse_kapanis.loc[hisse_kapanis["Close"] > hisse_kapanis["High"], "High"] = hisse_kapanis["Close"]
    hisse_kapanis.loc[hisse_kapanis["Close"] < hisse_kapanis["Low"], "Low"] = hisse_kapanis["Close"]

    df=pd.concat([df,hisse_kapanis],axis=0)
df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,High,Low,Open,Close,Volume,Hisse
Date,,,,,,
2023-06-15,27.500000,27.500000,27.500000,27.500000,262214.0,A1CAP.IS
2023-06-16,30.240000,30.240000,30.240000,30.240000,1169499.0,A1CAP.IS
2023-06-20,31.100000,29.940001,31.000000,29.940001,8064437.0,A1CAP.IS
2023-06-21,26.959999,26.959999,26.959999,26.959999,2147415.0,A1CAP.IS
2023-06-22,27.620001,25.500000,25.620001,25.940001,71898180.0,A1CAP.IS
...,...,...,...,...,...,...
2024-06-04,8.230000,7.750000,8.040000,7.950000,8406085.0,ZRGYO.IS
2024-06-05,8.070000,7.670000,7.970000,7.690000,2326506.0,ZRGYO.IS
2024-06-06,7.760000,7.530000,7.690000,7.540000,1654700.0,ZRGYO.IS


In [6]:
data=df.copy()

In [7]:
df["Close"]=np.round(df["Close"],2)
df["High"]=np.round(df["High"],2)
df["Low"]=np.round(df["Low"],2)
df["Open"]=np.round(df["Open"],2)
df["Volume"]=df["Volume"]*df["Close"]
df.columns=['High', 'Low', 'Open', 'Adj Close', 'Volume', 'Hisse']

In [8]:
canlı=pd.read_excel("./excel/FiyatPenceresi.Xlsx")
endeks=canlı.iloc[-1,:]
canlı=canlı.iloc[:-1,:]
canlı=canlı[["Yks","Dşk","Açl.Gün","Son.Fyt","Hcm","Kod","Lot"]]
canlı.columns=['High', 'Low', 'Open', 'Adj Close', 'Volume', 'Hisse',"Lot"]
canlı.index = pd.to_datetime(['2024-06-07'] * len(canlı))
canlı = canlı[~canlı['Hisse'].isin(['IDEAS', 'ISATR', 'ISKUR', 'ITTFH'])]
canlı["Lot"]=canlı["Lot"].astype("string")
canlı

,High,Low,Open,Adj Close,Volume,Hisse,Lot
2024-06-07,"27,80","26,42","26,52","26,62",43.092.452,A1CAP,1.600.912
2024-06-07,"132,50","128,70","131,90","130,00",9.438.996,ACSEL,72.371
2024-06-07,"520,00","480,00","515,50","491,00",163.571.749,ADEL,328.17
2024-06-07,"2,25","2,18","2,25","2,19",65.460.176,ADESE,29.506.155
2024-06-07,"27,50","26,60","27,48","26,98",27.539.137,ADGYO,1.020.693
...,...,...,...,...,...,...,...
2024-06-07,"6,34","6,06","6,34","6,10",19.280.136,YYAPI,3.107.976
2024-06-07,"11,97","11,74","11,94","11,74",43.262.498,YYLGD,3.647.588
2024-06-07,"12,14","11,04","11,05","12,14",136.648.839,ZEDUR,11.546.554
2024-06-07,"5,40","5,24","5,29","5,25",462.574.091,ZOREN,87.324.819


In [9]:
canlı=canlı.dropna()

In [10]:
for i in range(len(canlı)):
    if len(canlı.iloc[i, -1]) <= 7:
        if '.' in canlı.iloc[i, -1]:
            parts = canlı.iloc[i, -1].split('.')
            decimal_part = parts[1]
            if len(decimal_part) == 1:
                decimal_part += '00'
            elif len(decimal_part) == 2:
                decimal_part += '0'
            elif len(decimal_part) == 3:
                canlı.iloc[i, -1] = parts[0] + decimal_part[:2]
            canlı.iloc[i, -1] = parts[0] + decimal_part

In [11]:
kod=canlı["Hisse"].unique()
for i in range(len(kod)):
    kod[i]=kod[i]+".IS"

In [12]:
fark=np.setdiff1d(df["Hisse"].unique(),kod)

In [13]:
df = df[~df['Hisse'].isin(fark)]

In [14]:
canlı["Hisse"]=kod
canlı["High"]=canlı["High"].str.replace(",",".")
canlı["Low"]=canlı["Low"].str.replace(",",".")
canlı["Open"]=canlı["Open"].str.replace(",",".")
canlı["Adj Close"]=canlı["Adj Close"].str.replace(",",".")
canlı["Lot"]=canlı["Lot"].astype(str).str.replace(".", "")
canlı[["High","Low","Open","Adj Close","Lot"]]=canlı[["High","Low","Open","Adj Close","Lot"]].astype(float)
canlı["Volume"]=canlı["Lot"]*canlı["Adj Close"]
canlı

C:\Users\Bora\AppData\Local\Temp\ipykernel_2332\552829948.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canlı["Hisse"]=kod
C:\Users\Bora\AppData\Local\Temp\ipykernel_2332\552829948.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canlı["High"]=canlı["High"].str.replace(",",".")
C:\Users\Bora\AppData\Local\Temp\ipykernel_2332\552829948.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

,High,Low,Open,Adj Close,Volume,Hisse,Lot
2024-06-07,27.80,26.42,26.52,26.62,4.261628e+07,A1CAP.IS,1600912.0
2024-06-07,132.50,128.70,131.90,130.00,9.408230e+06,ACSEL.IS,72371.0
2024-06-07,520.00,480.00,515.50,491.00,1.611315e+08,ADEL.IS,328170.0
2024-06-07,2.25,2.18,2.25,2.19,6.461848e+07,ADESE.IS,29506155.0
2024-06-07,27.50,26.60,27.48,26.98,2.753830e+07,ADGYO.IS,1020693.0
...,...,...,...,...,...,...,...
2024-06-07,6.34,6.06,6.34,6.10,1.895865e+07,YYAPI.IS,3107976.0
2024-06-07,11.97,11.74,11.94,11.74,4.282268e+07,YYLGD.IS,3647588.0
2024-06-07,12.14,11.04,11.05,12.14,1.401752e+08,ZEDUR.IS,11546554.0
2024-06-07,5.40,5.24,5.29,5.25,4.584553e+08,ZOREN.IS,87324819.0


In [15]:
canlı=canlı[canlı["Hisse"].isin(df["Hisse"].loc["2024-06-07"])]

In [16]:
df["High"].loc["2024-06-07"]=canlı["High"].values
df["Low"].loc["2024-06-07"]=canlı["Low"].values
df["Open"].loc["2024-06-07"]=canlı["Open"].values
df["Adj Close"].loc["2024-06-07"]=canlı["Adj Close"].values
df["Volume"].loc["2024-06-07"]=canlı["Volume"].values

C:\Users\Bora\AppData\Local\Temp\ipykernel_2332\1307952629.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["High"].loc["2024-06-07"]=canlı["High"].values
C:\Users\Bora\AppData\Local\Temp\ipykernel_2332\1307952629.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Low"].loc["2024-06-07"]=canlı["Low"].values
C:\Users\Bora\AppData\Local\Temp\ipykernel_2332\1307952629.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Open

In [17]:
df["Kademe"] = np.where(df["Adj Close"] < 20, 0.01,
                np.where(df["Adj Close"] < 50, 0.02,
                    np.where(df["Adj Close"] < 100, 0.05,
                        np.where(df["Adj Close"] < 250, 0.1,
                            np.where(df["Adj Close"] < 500, 0.25,
                                np.where(df["Adj Close"] < 1000, 0.5,
                                    np.where(df["Adj Close"] < 2500, 1, 2.5)
                                )  # Buradaki parantez eksikti.
                            )
                        )
                    )
                )
            )

C:\Users\Bora\AppData\Local\Temp\ipykernel_2332\1526774384.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Kademe"] = np.where(df["Adj Close"] < 20, 0.01,


In [18]:
df["Tavan"]=(((df["Adj Close"].shift(1))*1.1)//df["Kademe"])*df["Kademe"]
df["Tavan"]=np.round(df["Tavan"],2)

C:\Users\Bora\AppData\Local\Temp\ipykernel_2332\134018246.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Tavan"]=(((df["Adj Close"].shift(1))*1.1)//df["Kademe"])*df["Kademe"]
C:\Users\Bora\AppData\Local\Temp\ipykernel_2332\134018246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Tavan"]=np.round(df["Tavan"],2)


In [19]:
df["Tavan Kontrol"]=np.where((df["Tavan"]==df["Adj Close"]),1,0)
df=df.drop(["Kademe","Tavan"],axis=1)

C:\Users\Bora\AppData\Local\Temp\ipykernel_2332\43100248.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Tavan Kontrol"]=np.where((df["Tavan"]==df["Adj Close"]),1,0)


In [20]:
import pandas_ta as ta

df["SMA 5"]=df["Adj Close"].rolling(5).mean()
df["SMA 10"]=df["Adj Close"].rolling(5).mean()
df["SMA 200"]=df["Adj Close"].rolling(5).mean()

# RSI
df["RSI"] = ta.rsi(df["Adj Close"],14)

# Bağıl Hacim
df["Bağıl Hacim"] = df["Volume"] / df["Volume"].rolling(10).mean()



# ADX, DMI, Aroon
adx_data = ta.adx(df["High"], df["Low"], df["Adj Close"], 14)
df["ADX"], df["DMIP"], df["DMIN"] = adx_data.iloc[:, 0], adx_data.iloc[:, 1], adx_data.iloc[:, 2]




def calculate_stoch_rsi(data, rsi_period=14, stochastic_period=14, k_period=3, d_period=3):
    rsi = ta.rsi(data["Adj Close"], rsi_period)
    stoch_rsi = (rsi - rsi.rolling(window=stochastic_period).min()) / (rsi.rolling(window=stochastic_period).max() - rsi.rolling(window=stochastic_period).min())
    stoch_rsi_k = stoch_rsi.rolling(window=k_period).mean() * 100
    stoch_rsi_d = stoch_rsi_k.rolling(window=d_period).mean()
    data['StochRSI_%K'] = stoch_rsi_k
    data['StochRSI_%D'] = stoch_rsi_d
    return data

df = calculate_stoch_rsi(df)


def calculate_macd(data, short_period=12, long_period=26, signal_period=9):
    data['EMA_12'] = ta.ema(data['Adj Close'], short_period)
    data['EMA_26'] = ta.ema(data['Adj Close'], long_period)
    data['MACD'] = data['EMA_12'] - data['EMA_26']
    data['Signal_Line'] = ta.ema(data['MACD'], signal_period)
    data['MACD_above_Signal'] = (data['MACD'] > data['Signal_Line']).astype(float)
    return data

df = calculate_macd(df)



# Diğer Göstergeler
df["DD"] = ((df["Adj Close"] / df["High"]) - 1) * 100
df["Range"] = ((df["High"] - df["Low"]) / df["Adj Close"]) * 100

df["MOM"] = ta.mom(df["Adj Close"], 10)



df["Driehaus Momentum"] = ((df["Bağıl Hacim"] > 2) & (df["RSI"] > 50) & (df["MOM"] > 1) & (df["Adj Close"] > df["SMA 5"]) & (df["Adj Close"] > df["SMA 10"]) & (df["Adj Close"] > df["SMA 200"])).astype(int)
df["MACD Yukarı Kesen"] = ((df["Bağıl Hacim"] > 1.5) & (df["MACD"] > 0) & (df["MACD_above_Signal"] == 1)).astype(int)
df["ADXDMI20"] = ((df["Bağıl Hacim"] > 1.3) & (df["ADX"] > 20) & (df["DMIP"] > df["DMIN"])).astype(int)
df["MACD+Stoch"] = ((df["Bağıl Hacim"] > 1.3) & (df["StochRSI_%K"] > df["StochRSI_%D"]) & (df["MACD_above_Signal"] == 1)).astype(int)




In [21]:
df["Return"]=100*((df["Open"]/df["Adj Close"].shift(2))-1)
df["Return"]=df["Return"].shift(-2)
df["Gerçek Return"]=100*((df["Open"]/df["Open"].shift(1))-1)
df["Gerçek Return"]=df["Gerçek Return"].shift(-2)
df["Date"]=pd.to_datetime(df.index)

In [22]:
endeks=endeks["Son.Fyt"]
endeks=endeks.replace(",",".")
endeks=float(endeks)

In [23]:
index=yf.download(tickers="XU100.IS",start="2000-01-01")
df["Endeks"]=index["Adj Close"]
df["Endeks Return"]=100*((df["Endeks"]/df["Endeks"].shift(1))-1)
df["Return Lag"]=((df["Adj Close"]/df["Adj Close"].shift(1))-1)*100
endeks_son=((endeks/index["Adj Close"].iloc[-2])-1)*100
df["Endeks Return"].loc["2024-06-07"]=endeks_son
df

[*********************100%%**********************]  1 of 1 completed
C:\Users\Bora\AppData\Local\Temp\ipykernel_2332\1358890307.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Endeks Return"].loc["2024-06-07"]=endeks_son


,High,Low,Open,Adj Close,Volume,Hisse,Tavan Kontrol,SMA 5,SMA 10,SMA 200,...,Driehaus Momentum,MACD Yukarı Kesen,ADXDMI20,MACD+Stoch,Return,Gerçek Return,Date,Endeks,Endeks Return,Return Lag
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-15,27.50,27.50,27.50,27.50,7.210885e+06,A1CAP.IS,0,NaN,NaN,NaN,...,0,0,0,0,12.727273,2.513228,2023-06-15,5495.500000,NaN,NaN
2023-06-16,30.24,30.24,30.24,30.24,3.536565e+07,A1CAP.IS,1,NaN,NaN,NaN,...,0,0,0,0,-10.846561,-13.032258,2023-06-16,5475.500000,-0.363934,9.963636
2023-06-20,31.10,29.94,31.00,29.94,2.414492e+08,A1CAP.IS,0,NaN,NaN,NaN,...,0,0,0,0,-14.428858,-4.970326,2023-06-20,5251.600098,-4.089122,-0.992063
2023-06-21,26.96,26.96,26.96,26.96,5.789431e+07,A1CAP.IS,0,NaN,NaN,NaN,...,0,0,0,0,-3.041543,2.029664,2023-06-21,5186.799805,-1.233915,-9.953240
2023-06-22,27.62,25.50,25.62,25.94,1.865039e+09,A1CAP.IS,0,28.116,28.116,28.116,...,0,0,0,0,-0.154202,-0.918133,2023-06-22,5428.299805,4.656050,-3.783383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-04,8.23,7.75,8.04,7.95,6.682838e+07,ZRGYO.IS,0,7.870,7.870,7.870,...,0,0,1,0,-3.270440,-3.513174,2024-06-04,10364.599609,-1.820629,-0.125628
2024-06-05,8.07,7.67,7.97,7.69,1.789083e+07,ZRGYO.IS,0,7.930,7.930,7.930,...,0,0,0,0,-0.520156,-0.520156,2024-06-05,10351.000000,-0.131212,-3.270440
2024-06-06,7.76,7.53,7.69,7.54,1.247644e+07,ZRGYO.IS,0,7.860,7.860,7.860,...,0,0,0,0,1.458886,0.000000,2024-06-06,10279.700195,-0.688820,-1.950585


In [24]:
bist100=pd.read_excel("./excel/bist100.xlsx")
bist100=bist100["Kod"].values
for i in range(len(bist100)):
    bist100[i]=bist100[i]+".IS"

In [25]:

yasak=["YYAPI","EMNIS","TETMT","RODRG","BRKO","AYES","EUKYO","YGYO","BALAT","SONME","SNKRN","KSTUR","DERIM","UZERB","MARKA","MMCAS","YAYLA"]
for i in range(len(yasak)):
    yasak[i]=yasak[i]+".IS"

In [26]:
df=df.dropna()

In [27]:
result = df.groupby('Hisse').apply(lambda x: x.iloc[2:])
result=result.droplevel(0)
result

,High,Low,Open,Adj Close,Volume,Hisse,Tavan Kontrol,SMA 5,SMA 10,SMA 200,...,Driehaus Momentum,MACD Yukarı Kesen,ADXDMI20,MACD+Stoch,Return,Gerçek Return,Date,Endeks,Endeks Return,Return Lag
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-08,46.14,42.02,45.80,42.70,5.898352e+08,A1CAP.IS,0,40.908,40.908,40.908,...,0,0,0,0,1.733021,1.023256,2023-08-08,7412.200195,-0.775089,-3.655235
2023-08-09,45.16,42.50,43.00,42.80,4.983311e+08,A1CAP.IS,0,42.112,42.112,42.112,...,0,0,0,0,-4.719626,-6.123389,2023-08-09,7600.700195,2.543105,0.234192
2023-08-10,44.90,39.50,43.44,40.68,4.606404e+08,A1CAP.IS,0,42.160,42.160,42.160,...,0,0,0,0,3.392330,3.138794,2023-08-10,7441.500000,-2.094546,-4.953271
2023-08-11,42.02,39.00,40.78,41.50,4.842361e+08,A1CAP.IS,0,42.400,42.400,42.400,...,0,0,0,0,-0.240964,-1.569187,2023-08-11,7714.399902,3.667270,2.015733
2023-08-14,43.26,40.94,42.06,41.10,4.254512e+08,A1CAP.IS,0,41.756,41.756,41.756,...,0,0,0,0,4.525547,3.768116,2023-08-14,7737.399902,0.298144,-0.963855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,8.20,7.53,7.91,8.16,1.656586e+08,ZRGYO.IS,0,7.566,7.566,7.566,...,0,1,1,1,-1.470588,-1.470588,2024-05-31,10400.500000,-0.770895,3.422053
2024-06-03,8.39,7.94,8.16,7.96,7.197300e+07,ZRGYO.IS,0,7.724,7.724,7.724,...,0,0,1,1,0.125628,-0.870647,2024-06-03,10556.799805,1.502810,-2.450980
2024-06-04,8.23,7.75,8.04,7.95,6.682838e+07,ZRGYO.IS,0,7.870,7.870,7.870,...,0,0,1,0,-3.270440,-3.513174,2024-06-04,10364.599609,-1.820629,-0.125628


In [36]:
result = df.groupby('Hisse').apply(lambda x: x.iloc[2:])
result=result.droplevel(0)
from sklearn.preprocessing import MinMaxScaler
result=result[result["Hisse"].isin(bist100)]
features=["Endeks Return","Return Lag","Range","RSI","Volume","Bağıl Hacim","DD","MACD+Stoch","Driehaus Momentum","ADXDMI20","MACD Yukarı Kesen","High","Low","Open","Adj Close"]
scaler1=MinMaxScaler(feature_range=(0,1))
result[features]=scaler1.fit_transform(result[features])
scaler2=MinMaxScaler(feature_range=(0,1))
result["Return"]=scaler2.fit_transform(np.array(result["Return"]).reshape(-1,1))
test=result.loc["2024-06-10"]
X_test=test[["Endeks Return","Return Lag","Range","RSI","Volume","Bağıl Hacim","DD","MACD+Stoch","Driehaus Momentum","ADXDMI20","MACD Yukarı Kesen","High","Low","Open","Adj Close"]]



In [37]:
params={'iterations': 571,
 'depth': 5,
 'learning_rate': 0.05514261151828995,
 'random_strength': 4,
 'bagging_temperature': 0.9572252022114495,
 'border_count': 59,
 'l2_leaf_reg': 20}
model=CatBoostRegressor(**params,task_type="GPU",random_seed=123)
model.load_model("./model/bist100scaling.bin")

In [38]:
tahminler=pd.DataFrame(scaler2.inverse_transform(np.array(test["Return"]).reshape(-1,1)))
tahminler=tahminler.set_index(X_test.index)
tahminler["Tahmin"]=scaler2.inverse_transform(np.array(model.predict(X_test)).reshape(-1,1))
tahminler.columns=["Gerçek","Tahmin"]
tahminler["Hisse"]=test["Hisse"]
tahminler["Tavan"]=test["Tavan Kontrol"]
tahminler=tahminler[tahminler["Tavan"]==0]
tahminler

,Gerçek,Tahmin,Hisse,Tavan
Date,,,,
2024-06-10,-99.561905,-1.876881,AEFES.IS,0
2024-06-10,-95.015576,-2.325954,AGROT.IS,0
2024-06-10,-54.129129,-2.078258,AHGAZ.IS,0
2024-06-10,-88.814130,-2.014693,AKBNK.IS,0
2024-06-10,-98.318182,-1.999890,AKCNS.IS,0
...,...,...,...,...
2024-06-10,-99.381953,-2.025689,VESTL.IS,0
2024-06-10,-99.976649,-1.994550,YEOTK.IS,0
2024-06-10,-91.493384,-2.203911,YKBNK.IS,0


In [39]:
top_5_rows = tahminler.groupby(level=0).apply(lambda x: x.nlargest(3, columns=['Tahmin'])).dropna()
top_5_rows=top_5_rows.droplevel(level=0)

In [40]:
indexler=df.index.unique()
tahminler=tahminler.set_index(pd.to_datetime(["2024-06-06"]*len(tahminler)))

In [41]:
window_size=10
vars=[]
tahminler["Var"]=0
for index, group in top_5_rows.groupby(level=0):
    hisseler=group["Hisse"]
    for hisse in hisseler:
        try:
           window_data=df[df["Hisse"]==hisse]["Adj Close"].loc[indexler[-500]:].loc[:index].iloc[-window_size:]
           getiriler=window_data.pct_change(window_size-1)*100
           getiri=getiriler.iloc[-1]
           std=window_data.std()
           var=getiri/std
           vars.append(var)
           top_5_rows.loc[(top_5_rows["Hisse"]==hisse) & (top_5_rows.index==index), "Var"] = var
        except Exception as e:
            print(e)
top_5_rows

,Gerçek,Tahmin,Hisse,Tavan,Var
Date,,,,,
2024-06-10,266.336634,-1.487748,REEDR.IS,0,2.711710
2024-06-10,-96.632911,-1.711826,TURSG.IS,0,2.443014
2024-06-10,-28.414097,-1.723742,TABGD.IS,0,0.402144


In [42]:
orj_test=scaler1.inverse_transform(X_test)
orj_test=pd.DataFrame(orj_test,columns=X_test.columns)
orj_test=orj_test.set_index(X_test.index)
orj_test["Hisse"]=test[test["Hisse"].isin(bist100)]["Hisse"].values



In [43]:
bakiye=4051.02
veri=top_5_rows[top_5_rows["Var"]>0]
veri=veri.head(3)
veri=veri.sort_values(by="Hisse",ascending=True)
veri['Ağırlık'] = veri['Var'] / veri.groupby(level=0)['Var'].transform('sum')
veri["Fiyat"]=orj_test[orj_test["Hisse"].isin(veri["Hisse"])].sort_values(by="Hisse",ascending=True)["Adj Close"].values
veri["Para"]=bakiye*veri["Ağırlık"]
veri["Lot"]=veri["Para"]//(veri["Fiyat"]*1.1)
veri

,Gerçek,Tahmin,Hisse,Tavan,Var,Ağırlık,Fiyat,Para,Lot
Date,,,,,,,,,
2024-06-10,266.336634,-1.487748,REEDR.IS,0,2.711710,0.487992,40.4,1976.867006,44.0
2024-06-10,-28.414097,-1.723742,TABGD.IS,0,0.402144,0.072369,181.6,293.167712,1.0
2024-06-10,-96.632911,-1.711826,TURSG.IS,0,2.443014,0.439639,79.0,1780.985282,20.0
